# 一、核心思想
&nbsp;&nbsp;&nbsp;&nbsp; 只能用于<b>二分图</b>。在当前图划分的基础上，通过尝试交换两个子集之间的节点，寻找使两个子集间边数减少的方案。
# 二、算法流程
1. 定义增益：设节点$a∈A、b∈B$，则交换增益$g(a,b)=D(a)+D(b)-2*w(a,b)$。其中$D(a)$表示节点a的外部度（从a到B的权总和）减去内部度（从 a 到 A 的边权重和）；$w(a,b)$表示若a和b之间有边，则是边的权总。因此，$g(a,b)$可看作交换a和b之后，边的减少量。
2. 初始化社区A和B。
3. 计算所有可能的交换对$(a_i,b_i )$，选择$g(a_i, b_i)$最大的作为当前交换对（即使为负也选）。并锁定$(a_i, b_i)$，后续不再考虑。
4. 重复步骤③，得到一系列交换对$(a_1,b_1 )、(a_2,b_2)$...
5. 计算累计增益$G=∑g$，选择使G最大的前k个交换对。
# 三、初始化
1. 随机初始化为两个社区
2. 基于节点度划分，将将高度数的几点分开，避免他们初始时在一起造成不均衡
3. 贪心分配，先将两个节点分配到两个社区，然后对每个节点，放入使外部度增加最小的那个社区。
# 四、缺点
1. 初始化影响大，尽管利用了类似模拟退火的思想，但还是容易收敛到局部。
2. 在大图上，运算效率慢。假设不预先存储D，则每轮迭代时间复杂度为$O(n^2)$，共需迭代$O(n/2)$次。存储了D，每轮迭代的时间复杂度为$O(n)$。
3. 要求两个社区节点数相等或给定，<b>不适用于大小未知的社区发现</b>。若要发现多个社区，需迭代子图。


In [11]:
import numpy as np
import itertools

In [29]:
def KL(graph):
    '''
    graph: 邻接矩阵
    '''

    # 1. 基于节点度初始化
    degree = np.sum(graph, axis = 0)
    idx = degree.argsort() # 排序后，奇数和偶数的节点分开
    C1 = idx[::2]
    C2 = idx[1::2]
    # 2. KL算法
    ## 1) 先计算每个节点对于C1、C2两个社区的度
    weights = np.zeros((len(graph), 2))
    weights[:, 0] = np.sum(graph[:, C1], axis=1)
    weights[:, 1] = np.sum(graph[:, C2], axis=1)

    C1 = set(C1)
    C2 = set(C2)
    ## 2) 再迭代计算每个交换对的增益
    fixed = set() # 存储锁定节点
    g_series = [] # 存储g的序列值
    exchange = []
    for i in range(min(len(C1), len(C2))):
        product = np.array(list(itertools.product([x for x in C1 if x not in fixed], [x for x in C2 if x not in fixed])))
        a_idx = product[:, 0]
        b_idx = product[:, 1]
        g = (weights[a_idx,1]-weights[a_idx,0]) + (weights[b_idx,0]-weights[b_idx,1]) - 2 * graph[a_idx,b_idx] # g(a,b) = D(a)+D(b)-2*w(a,b)
        max_idx = np.argmax(g) # 寻找最大的g
        g_series.append(g[max_idx])
        fixed.update(product[max_idx])
        a, b = product[max_idx]
        weights[:, 0] += (graph[:, b] - graph[:, a]) # 更新节点度
        weights[:, 1] += (graph[:, a] - graph[:, b])
        C1.discard(a)
        C2.discard(b)
        C1.add(b)
        C2.add(a)
        exchange.append((a,b))

    ## 3) 去顶累计最大G，并使用前k个交换对
    cum_g = np.cumsum(g_series)
    G_idx = np.argmax(cum_g)
    for i in range(len(exchange)-1, G_idx-1, -1):
        a, b = exchange[i]
        C1.add(a)
        C2.add(b)
        C1.discard(b)
        C2.discard(a)

    return G_idx, C1, C2

graph = np.array([
    [0, 1, 1, 0, 1],
    [1, 0, 1, 0, 1],
    [1, 1, 0, 1, 1],
    [0, 0, 1, 0, 0],
    [1, 1, 1, 0, 0],
])
KL(graph)

(np.int64(0),
 {np.int64(1), np.int64(2), np.int64(3)},
 {np.int64(0), np.int64(4)})